# Classifying different workouts

In [ ]:
import logging
import os
from datetime import date, timedelta

from dotenv import load_dotenv
from qself.oura import OuraAPIClient

In [ ]:
logging.basicConfig(level=logging.INFO)
load_dotenv()

In [ ]:
client = OuraAPIClient(os.environ["OURA_PERSONAL_ACCESS_TOKEN"])

In [ ]:
end = date.today()
start = end - timedelta(days=6)

In [ ]:
workouts = client("workout", start.isoformat(), (end + timedelta(days=1)).isoformat())
workouts